In [1]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn

from mindnlp import load_dataset

from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy

/home/daiyuxin/anaconda3/envs/mindspore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

In [3]:
imdb_train.get_dataset_size()

25000

In [4]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=16, shuffle=False):
    def tokenize(text):
        tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized[0], tokenized[1]

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})

    return dataset

In [5]:
from mindnlp.transformers import GPTTokenizer
# tokenizer
gpt_tokenizer = GPTTokenizer.from_pretrained('openai-gpt', from_pt=True)

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [6]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [7]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [8]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[16, 466], dtype=Int64, value=
 [[  481, 14863,   544 ... 40480, 40480, 40480],
  [  507,   544,   595 ... 40480, 40480, 40480],
  [ 2770,  2348,   651 ... 40480, 40480, 40480],
  ...
  [  645,  2476,  7046 ... 40480, 40480, 40480],
  [ 7699,   616,  5011 ... 40480, 40480, 40480],
  [ 1099,  1098,   500 ... 40480, 40480, 40480]]),
 Tensor(shape=[16, 466], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[16], dtype=Int32, value= [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1])]

In [9]:
from mindnlp.transformers import GPTForSequenceClassification

# set bert config and define parameters for training
model = GPTForSequenceClassification.from_pretrained('openai-gpt', from_pt=True, num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

metric = Accuracy()

# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt_imdb_finetune', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='gpt_imdb_finetune_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_train, metrics=metric,
                  epochs=3, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)

The following parameters in models are missing parameter:
['score.weight']


In [10]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0: 100%|█████████████████████████████████████████████████████████████████████| 1094/1094 [13:11<00:00,  1.38it/s, loss=0.24947281]


Checkpoint: 'gpt_imdb_finetune_epoch_0.ckpt' has been saved in epoch: 0.


Evaluate: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1094/1094 [06:04<00:00,  3.00it/s]


Evaluate Score: {'Accuracy': 0.9548571428571428}
---------------Best Model: 'gpt_imdb_finetune_best.ckpt' has been saved in epoch: 0.---------------


...

Evaluate Score: {'Accuracy': 0.9726285714285714}
---------------Best Model: 'gpt_imdb_finetune_best.ckpt' has been saved in epoch: 1.---------------


Epoch 2: 100%|█████████████████████████████████████████████████████████████████████| 1094/1094 [13:08<00:00,  1.39it/s, loss=0.10463984]


The maximum number of stored checkpoints has been reached.
Checkpoint: 'gpt_imdb_finetune_epoch_2.ckpt' has been saved in epoch: 2.


Evaluate: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1094/1094 [06:03<00:00,  3.01it/s]


Evaluate Score: {'Accuracy': 0.9845142857142857}
---------------Best Model: 'gpt_imdb_finetune_best.ckpt' has been saved in epoch: 2.---------------
Loading best model from 'checkpoint' with '['Accuracy']': [0.9845142857142857]...
---------------The model is already load the best model from 'gpt_imdb_finetune_best.ckpt'.---------------


In [11]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="labels")

Evaluate: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [08:33<00:00,  3.04it/s]

Evaluate Score: {'Accuracy': 0.93028}
